# Explore 311 Requests

This notebook is a simplified companion to bulky-items-choropleth.ipynb.  It can be used to select a request type and visualize with the comparison choropleths and the animated heat map.  It is a bit mechanical for starters.

In [ ]:
import pandas as pd
import geopandas as gpd

from ipywidgets import Output, HBox, interactive

from collections import defaultdict, OrderedDict

import sys
sys.path.append('../src')

from lasan import AnimatedMap, nc_choropleth

# Data Setup

I processed the data and transformed to parquet files.  They seem to be quite efficient and fast.

This section covers the two sources used:

1. The 311 data is a subset of the entire 2021 data.  After I read it, I show a couple of summary charts/statistics.  Finally I create a specific subset for Bulky Items.  **Note:** Now we may need to focus on Illegal Dumping?  Will need to refactor a bit so it can be parameterized for the choropleths.
2. The Neighorhood Council data is polygon data from data.lacity.org.  I add the 311 counts per NC for the first choropleth.  After that I use polygon information to compute square miles for the NC and use that for the density.

In [ ]:
%%time
thin_gdf = gpd.read_parquet('../data/thin2.parq')

neighborhoods_gdf = gpd.read_parquet('../data/nc.parq')

This shows the distribution of request types for 2021.

In [ ]:
thin_gdf.request_type.value_counts().plot.barh(figsize=(8, 4));

Use the chart above to identify a request type to explore.  Change the request_type variable in the next cell.

**Note:** I combined cells so once you select a different request_type it is simple to execute the next two cells to see the results.

In [ ]:
request_type = 'Illegal Dumping Pickup'
request_gdf = thin_gdf.query(f"request_type == @request_type").reset_index().drop(columns=['index'])

request_counts = request_gdf['nc'].value_counts().to_frame().reset_index().rename(columns={'index': 'nc_id', 'nc': 'count'})

request_merged_gdf = pd.merge(neighborhoods_gdf, request_counts, how="left", on=["nc_id"])

request_gdf['day'] = request_gdf['created_dt'].apply(lambda dt: dt.date().strftime("%m-%d (%A)"))

from pyproj import Geod

geod = Geod(ellps="WGS84")

def square_miles(geo):
    square_meters = abs(geod.geometry_area_perimeter(geo)[0])
    return (square_meters * 10.764) / 27878000

request_merged_gdf['sq_miles'] = request_merged_gdf.apply(lambda row: square_miles(row.geometry), axis=1)

request_merged_gdf['density'] = request_merged_gdf.apply(lambda row: round((row['count'] / row['sq_miles']), 2) , axis=1)

choropleth_map = nc_choropleth(request_merged_gdf, "count", ['nc_id','count'])
density_choropleth_map = nc_choropleth(request_merged_gdf, "density", ['nc_id','density'])


count_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

density_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

with count_output:
    display(choropleth_map)

with density_output:
    display(density_choropleth_map)

print('\nMaps for request type: ' + request_type + '\n\n')
HBox([count_output, density_output])

# Animated Heat Map

The final map visualization.  

  1. The 311 calls have be entended with day of the week for the timeline.
  2. The NC polygons provide context and map navigation.
  
Pretty basic use of the HeatMapWithTime plugin from folium

I've wrapped the display in the interactive widget.  The pick list is sorted by count.

In [ ]:
an_map = AnimatedMap(observations=request_gdf, boundaries=neighborhoods_gdf)

def silly(nc_name):
    an_map.nc(nc_name)
    display(an_map.nc_hmap)

w = interactive(silly, nc_name=request_merged_gdf.sort_values(by=['count'], ascending=False).name.to_list())

w